### Ingestas desde archivos CSV a la capa de Bronce en formato .parquet
Se utilizan las clases para poder realizar las ingestas con Spark y/o Pandas

In [11]:
from pyspark.sql import SparkSession
from medallion_pkg.dataLoader import SparkDataLoader
from medallion_pkg.dataLoader import PandasDataLoader

Carga el archivo csv a parquet utilizando la clase SparkDataLoader

In [13]:
spark_loader_covid_19_clean_complete = SparkDataLoader('/opt/spark/scripts/source/covid_19_clean_complete.csv', '/opt/spark/scripts/data/bronce/covid_19_clean_complete')
spark_loader_covid_19_clean_complete.load_and_save_parquet()
rdd = spark_loader_covid_19_clean_complete.load_csv_as_rdd()
#  Leer el parquet ingestado
spark = SparkSession.builder \
    .appName("Read parquet") \
    .getOrCreate()
df_bronce = spark.read.parquet("/opt/spark/scripts/data/bronce/covid_19_clean_complete")
df_bronce.show(truncate = False)

25/07/22 19:22:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
                                                                                

Spark guardó el DataFrame como Parquet en: /opt/spark/scripts/data/bronce/covid_19_clean_complete


RDD count: 49068


25/07/22 19:22:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------------------------+-------------------+---------+----------+----------+---------+------+---------+------+---------------------+
|Province/State              |Country/Region     |Lat      |Long      |Date      |Confirmed|Deaths|Recovered|Active|WHO Region           |
+----------------------------+-------------------+---------+----------+----------+---------+------+---------+------+---------------------+
|NULL                        |Afghanistan        |33.93911 |67.709953 |2020-01-22|0        |0     |0        |0     |Eastern Mediterranean|
|NULL                        |Albania            |41.1533  |20.1683   |2020-01-22|0        |0     |0        |0     |Europe               |
|NULL                        |Algeria            |28.0339  |1.6596    |2020-01-22|0        |0     |0        |0     |Africa               |
|NULL                        |Andorra            |42.5063  |1.5218    |2020-01-22|0        |0     |0        |0     |Europe               |
|NULL                      

Carga el archivo csv a parquet utilizando la clase PandasDataLoader

In [ ]:
pandas_loader = PandasDataLoader('/opt/spark/scripts/source/usa_county_wise.csv', '/opt/spark/scripts/data/bronce/usa_county_wise.parquet')
pandas_loader.load_and_save_parquet()

Esto sería una alternativa para mantener la ingesta parametrizada en un archivo; de esta manera se puede ir agregando mas input a las ingestas mediante este archivo

In [ ]:
spark = SparkSession.builder \
    .appName("CSV to Parquet") \
    .getOrCreate()

path_file = "/opt/spark/scripts/source/paths_ingesta.txt"
rdd_paths = spark.sparkContext.textFile(path_file)

# Parsear las rutas
paths = rdd_paths.map(lambda line: line.strip().split(",")) \
                 .filter(lambda x: len(x) == 2) \
                 .collect()

# Procesar cada archivo
for origen, destino in paths:
    print(f"Procesando: {origen} → {destino}")
    spark_loader = SparkDataLoader(origen, destino)
    spark_loader.load_and_save_parquet()